# Assignment 3a: Basic Gradio RAG Frontend
## Day 6 Session 2 - Building Simple RAG Applications

In this assignment, you'll build a simple Gradio frontend for your RAG system with just the essential features:
- Button to initialize the vector database
- Search query input and button
- Display of AI responses

**Learning Objectives:**
- Create basic Gradio interfaces
- Connect RAG backend to frontend
- Handle user interactions and database initialization
- Build functional AI-powered web applications

**Prerequisites:**
- Completed Assignment 1 (Vector Database Basics)
- Completed Assignment 2 (Advanced RAG)
- Understanding of LlamaIndex fundamentals


## 📚 Part 1: Setup and Imports

Import all necessary libraries for building your Gradio RAG application.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Get OPENROUTER_API_KEY
from google.colab import userdata
import os

os.environ["OPENROUTER_API_KEY"] = userdata.get('OPENROUTER_API_KEY')

Mounted at /content/drive


In [2]:
!pip install -r "/content/drive/MyDrive/Outskill-RAG/requirements.txt" --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 41.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.7/38.7 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 122.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 104.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 7.0 MB/s eta 0:00

In [3]:
# Import required libraries
import gradio as gr
import os
from pathlib import Path

# LlamaIndex components
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, Settings
from llama_index.vector_stores.lancedb import LanceDBVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openrouter import OpenRouter

print("✅ All libraries imported successfully!")


✅ All libraries imported successfully!


## 🤖 Part 2: RAG Backend Class

Create a simple RAG backend that can initialize the database and answer queries.


In [11]:
class SimpleRAGBackend:
    """Simple RAG backend for Gradio frontend."""

    def __init__(self):
        self.index = None
        self.setup_settings()
        print("Backend initialized with settings.")

    def setup_settings(self):
        """Configure LlamaIndex settings."""
        # Set up the LLM using OpenRouter
        api_key = os.getenv("OPENROUTER_API_KEY")
        if api_key:
            Settings.llm = OpenRouter(
                api_key=api_key,
                model="gpt-4o",
                temperature=0.1
            )

        # Set up the embedding model
        Settings.embed_model = HuggingFaceEmbedding(
            model_name="BAAI/bge-small-en-v1.5",
            trust_remote_code=True
        )

        # Set chunking parameters
        Settings.chunk_size = 512
        Settings.chunk_overlap = 50
        print(f"Chunk size set to {Settings.chunk_size} with overlap {Settings.chunk_overlap}.")


    def initialize_database(self, data_folder="/content/drive/MyDrive/Outskill-RAG/content/papers/agents"):
        """Initialize the vector database with documents."""
        print(f"Attempting to initialize database from: {data_folder}")
        # Check if data folder exists
        if not Path(data_folder).exists():
            print(f"❌ Data folder '{data_folder}' not found!")
            return f"❌ Data folder '{data_folder}' not found!"

        try:
            # Create vector store
            print(f"Creating LanceDB vector store at: /content/storage/basic_rag_vectordb")
            vector_store = LanceDBVectorStore(
                uri="/content/storage/basic_rag_vectordb",
                table_name="documents"
            )
            print("LanceDB vector store created.")

            # Load documents
            print(f"Loading documents from {data_folder}...")
            reader = SimpleDirectoryReader(input_dir=data_folder, recursive=True)
            documents = reader.load_data()
            print(f"Loaded {len(documents)} documents.")


            # Create storage context and index
            print("Creating storage context and vector index...")
            storage_context = StorageContext.from_defaults(vector_store=vector_store)
            self.index = VectorStoreIndex.from_documents(
                documents,
                storage_context=storage_context,
                show_progress=True
            )
            print("Vector index created.")

            return f"✅ Database initialized successfully with {len(documents)} documents!"

        except Exception as e:
            print(f"❌ Error during database initialization: {str(e)}")
            return f"❌ Error initializing database: {str(e)}"

    def query(self, question):
        """Query the RAG system and return response."""
        print(f"Received query: {question}")
        # Check if index exists
        if self.index is None:
            print("❌ Index not initialized.")
            return "❌ Please initialize the database first!"

        # Check if question is empty
        if not question or not question.strip():
            print("⚠️ Empty question received.")
            return "⚠️ Please enter a question first!"

        try:
            # Create query engine and get response
            print("Creating query engine and querying index...")
            query_engine = self.index.as_query_engine()
            response = query_engine.query(question)
            print("Query successful.")
            return str(response)

        except Exception as e:
            print(f"❌ Error processing query: {str(e)}")
            return f"❌ Error processing query: {str(e)}"

# Initialize the backend
data_folder = "/content/drive/MyDrive/Outskill-RAG/content/papers/agents"
os.makedirs("/content/storage/basic_rag_vectordb", exist_ok=True)
rag_backend = SimpleRAGBackend()
# rag_backend.initialize_database(data_folder)
print("🚀 RAG Backend initialized and ready!")

Chunk size set to 512 with overlap 50.
Backend initialized with settings.
🚀 RAG Backend initialized and ready!


## 🎨 Part 3: Gradio Interface

Create a simple Gradio interface with:
1. Button to initialize the database
2. Text input for queries
3. Button to submit queries
4. Text output for responses
5. Text output for status messages


In [12]:
def create_basic_rag_interface():
    """Create basic RAG interface with essential features."""

    def initialize_db():
        """Handle database initialization."""
        return rag_backend.initialize_database(data_folder=data_folder)

    def handle_query(question):
        """Handle user queries."""
        return rag_backend.query(question)

    # DONE: Create Gradio interface using gr.Blocks()
    # Hint: Look at the structure below and fill in the missing components

    with gr.Blocks(title="Basic RAG Assistant") as interface:
        # DONE: Add title and description
        # Hint: Use gr.Markdown() for formatted text
        gr.Markdown("# Basic RAG Assistant")
        gr.Markdown("Build a simple Gradio frontend for your RAG system with essential features.")

        # DONE: Add initialization section
        # Hint: You need to use gr.Button to initialize the database
        init_btn = gr.Button("Initialize Database")

        # DONE: Add status output
        # Hint: You need to use gr.Textbox to display the status
        status_output = gr.Textbox(label="Status", interactive=False)

        # The connection between the button and the status output has already been implemented
        # at the end of this function

        # DONE: Add query section
        # Hint: You need a text input, submit button, and response output

        # Use gr.Textbox to create a text input
        query_input = gr.Textbox(label="Enter your question here:", placeholder="Type your question...")

        # Use gr.Button to create a submit button
        submit_btn = gr.Button("Ask Question")

        # Use gr.Textbox to create a response output
        response_output = gr.Textbox(label="AI Response", interactive=False, lines=10)

        # Connect buttons to functions
        # Uncomment when above is implemented
        init_btn.click(initialize_db, outputs=[status_output])
        submit_btn.click(handle_query, inputs=[query_input], outputs=[response_output])

    return interface

# Create the interface
basic_interface = create_basic_rag_interface()
print("✅ Basic RAG interface created successfully!")

✅ Basic RAG interface created successfully!


## 🚀 Part 4: Launch Your Application

Launch your Gradio application and test it!


In [13]:
print("🎉 Launching your Basic RAG Assistant...")
print("🔗 Your application will open in a new browser tab!")
print("")
print("📋 Testing Instructions:")
print("1. Click 'Initialize Database' button first")
print("2. Wait for success message")
print("3. Enter a question in the query box")
print("4. Click 'Ask Question' to get AI response")
print("")
print("💡 Example questions to try:")
print("- What are the main topics in the documents?")
print("- Summarize the key findings")
print("- Explain the methodology used")
print("")
print("🚀 Launch your app:")

# Your launch code here:
# Uncomment when implemented
basic_interface.launch(share=False)

🎉 Launching your Basic RAG Assistant...
🔗 Your application will open in a new browser tab!

📋 Testing Instructions:
1. Click 'Initialize Database' button first
2. Wait for success message
3. Enter a question in the query box
4. Click 'Ask Question' to get AI response

💡 Example questions to try:
- What are the main topics in the documents?
- Summarize the key findings
- Explain the methodology used

🚀 Launch your app:
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

## ✅ Assignment Completion Checklist

Before submitting, ensure you have:

- [x] RAG backend is provided and working
- [x] Created Gradio interface with required components:
  - [x] Title and description using gr.Markdown()
  - [x] Initialize database button using gr.Button()
  - [x] Status output using gr.Textbox()
  - [x] Query input field using gr.Textbox()
  - [x] Submit query button using gr.Button()
  - [x] Response output area using gr.Textbox()
- [x] Connected buttons to backend functions using .click()
- [x] Successfully launched the application
- [x] Tested the full workflow (initialize → query → response)

## 🎊 Congratulations!

You've successfully built your first Gradio RAG application! You now have:

- A functional web interface for your RAG system
- Understanding of Gradio basics and component connections
- A foundation for building more complex AI applications

**Next Steps**: Complete Assignment 3b to add advanced configuration options to your RAG interface!
